In [1]:
import os, sys
from sklearn.pipeline import Pipeline
from pathlib import Path

# Add local codebase to path
user = str(Path.home()).split('/')[-1]  # Slightly ugly way to get the username.
sys.path.insert(1, f'/data/{user}/Documents/woonfraude/codebase')

# Import own modules
from datasets_oo import *
from clean_oo import *
from extract_features_oo import *

SyntaxError: EOL while scanning string literal (datasets_oo.py, line 584)

In [ ]:
# bagDataset = BagDataset()
# bagDataset.download(force=True)
# bagDataset.load('download')
# bagDataset.bag_fix()

In [ ]:
# adresDataset = AdresDataset()
# adresDataset.load('download_leegstand_woningId_bag_personen_hotline')

In [ ]:
###################
## Load datasets ##
###################
# Download (or load cached versions of) the datasets.

# Initialize dataset objects
adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()
personenDataset = PersonenDataset()
bagDataset = BagDataset()
hotlineDataset = HotlineDataset()

# # Download and perform dataset-specific processing.
# # Uncomment code and run once. Use load-from cache functionality below afterwards.
# adresDataset.download(force=True)
# adresDataset.load('download')
# adresDataset.extract_leegstand()
# adresDataset.enrich_with_woning_id()

# zakenDataset.download(force=True)
# zakenDataset.load('download')
# zakenDataset.add_categories()
# zakenDataset.filter_categories()

# stadiaDataset.download(force=True)
# stadiaDataset.load('download')
# stadiaDataset.add_zaak_stadium_ids()
# stadiaDataset.add_labels()

# personenDataset.download(force=True)

# bagDataset.download(force=True)
# bagDataset.load('download')
# bagDataset.bag_fix()

# hotlineDataset.download(force=True)
# hotlineDataset.load('download')

# Load datasets from cache (if download and processing steps have already been done).
adresDataset.load('download_leegstand_woningId')
zakenDataset.load('download_categories_filterCategories')
stadiaDataset.load('download_ids_labels')
personenDataset.load('download')
bagDataset.load('download_columnFix')
hotlineDataset.load('download')

In [5]:
#########################
## Clean zaken dataset ##
#########################

zakenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=zakenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum','einddatum', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['afg_code_beh', 'beh_code', 'eigenaar', 'categorie'])
    )
    ])

zaken = zakenPipeline.fit_transform(zakenDataset.data)

Dataframe "zaken": Dropped 0 duplicates!
Dataframe "zaken": Fixed dates!
Dataframe "zaken": Cleaned out 3 dates!
Lowered strings of cols ['beh_code', 'beh_oms', 'afg_code_beh', 'afs_code', 'afs_oms', 'afg_code_afs', 'eigenaar', 'zaak_id', 'mededelingen', 'categorie'] in df zaken!
Now extracting features from column: 'afg_code_beh'.
Done!
Now extracting features from column: 'beh_code'.
Done!
Now extracting features from column: 'eigenaar'.
Done!
Now extracting features from column: 'categorie'.
Done!


In [6]:
##########################
## Clean stadia dataset ##
##########################

stadiaPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=stadiaDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum', 'peildatum', 'einddatum', 'date_created',
                          'date_modified', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

stadia = stadiaPipeline.fit_transform(stadiaDataset.data)

Dataframe "stadia": Dropped 0 duplicates!
Dataframe "stadia": Fixed dates!
Dataframe "stadia": Cleaned out 0 dates!
Lowered strings of cols ['afg_co', 'sta_code', 'sta_oms', 'afg_code_stad', 'afs_code', 'afs_oms', 'afg_code_afs', 'resultaat', 'mdr_code', 'user_created', 'user_modified', 'stadia_id', 'zaak_id', 'stadium_id', 'label'] in df stadia!


In [7]:
############################
## Clean personen dataset ##
############################

personenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=personenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['geboortedatum'],
        lower_string_columns=True)
    )])

personen = personenPipeline.fit_transform(personenDataset.data)

Dataframe "personen": Dropped 0 duplicates!
Dataframe "personen": Fixed dates!
Lowered strings of cols ['pen_type', 'gezinsverhouding', 'geslacht', 'voorletters', 'burgerlijke_staat', 'naam', 'geheim_adres', 'voorv_mnaam', 'voorv_naam', 'meisjesnaam', 'vertrekdatum_adam', 'ind_naamgebruik', 'nat_ned', 'ind_nat_ovlp', 'verblijfstatus', 'datum_einde_vblstat', 'landcode', 'user_created', 'user_modified', 'datum_begin_vblstat', 'ais_nr', 'crv_nr', 'geheim', 'in_onderzoek', 'datum_verkrijging_vreemd', 'voorletters_zdia', 'naam_zdia', 'voorv_mnaam_zdia', 'voorv_naam_zdia', 'meisjesnaam_zdia', 'nm_dia_255', 'mnm_dia_255'] in df personen!


In [8]:
#######################
## Clean BAG dataset ##
#######################

bagPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=bagDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=[],
        drop_columns = ['indicatie_geconstateerd', 'indicatie_in_onderzoek', 'woningvoorraad'],
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_mode=['status_coordinaat_code'],
#         impute_missing_values_mode=['status_coordinaat_code', 'indicatie_geconstateerd',
#                                     'indicatie_in_onderzoek', 'woningvoorraad'],
        fillna_columns={'_huisnummer_verblijfsobject': 0,
                         '_huisletter_verblijfsobject': 'None',
                         '_openbare_ruimte_naam_verblijfsobject': 'None',
                         '_huisnummer_toevoeging_verblijfsobject': 'None',
                         'type_woonobject_omschrijving': 'None',
                         'eigendomsverhouding_id': 'None',
                         'financieringswijze_id': -1,
                         'gebruik_id': -1,
                         'reden_opvoer_id': -1,
                         'status_id_verblijfsobject': -1,
                         'toegang_id': 'None'})
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['status_coordinaat_code', 'type_woonobject_omschrijving',
                              'eigendomsverhouding_id', 'financieringswijze_id',
                              'gebruik_id', 'ligging_id', 'reden_opvoer_id',
                              'status_id_nummeraanduiding', 'toegang_id'])
    )
    ])

bag = bagPipeline.fit_transform(bagDataset.data)

Dataframe "bag": Dropped 0 duplicates!
Lowered strings of cols ['id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'postcode', 'type', 'adres_nummer', 'vervallen_nummeraanduiding', 'hoofdadres', '_openbare_ruimte_naam_nummeraanduiding', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'id_ligplaats', 'landelijk_id_ligplaats', 'vervallen_ligplaats', 'geometrie_ligplaats', '_huisletter_ligplaats', 'bron_id_ligplaats', 'status_id_ligplaats', 'id_standplaats', 'landelijk_id_standplaats', 'vervallen_standplaats', 'geometrie_standplaats', '_huisnummer_toevoeging_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'id_verblijfsobject', 'landelijk_id_verblijfsobject', 'status_coordinaat_code', 'status_coordinaat_omschrijving', 'type_woonobject_code', 'type_woonobject_omschrijving', 'geometrie_verblijfsobject', '_openbare_ruimte_naam_verblijfso

In [9]:
###########################
## Clean hotline dataset ##
###########################

hotlinePipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=hotlineDataset.id_column,
        drop_duplicates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

hotline = hotlinePipeline.fit_transform(hotlineDataset.data)

Dataframe "hotline": Dropped 0 duplicates!
Lowered strings of cols ['mdw_code', 'overtreding_code', 'melder_anoniem', 'melder_naam', 'melder_emailadres', 'melder_telnr', 'situatie_schets', 'user_created', 'user_modified'] in df hotline!


In [10]:
#########################
## Clean adres dataset ##
#########################

# Hier de extract stap weghalen? Deze past waarschijnlijk beter na het combinen v/d datasets.
adresPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=adresDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['hvv_dag_tek', 'max_vestig_dtm', 'wzs_update_datumtijd'],
        lower_string_columns=True,
        impute_missing_values=True,
        fillna_columns={'hsnr': 0, 'sttnaam': 'None', 'hsltr': 'None', 'toev': 'None'})
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr'],
        ))
    ])

adres = adresPipeline.fit_transform(adresDataset.data)

Dataframe "adres": Dropped 0 duplicates!
Dataframe "adres": Fixed dates!
Lowered strings of cols ['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'hvv_dag_tek', 'max_vestig_dtm'] in df adres!
Now extracting features from column: 'toev'.
Done!
Now extracting features from column: 'pvh_omschr'.
Done!
Now extracting features from column: 'sbw_omschr'.
Done!
Now extracting features from column: 'sbv_

In [11]:
######################
## Combine datasets ##
######################

# TODO: - UPDATEN VAN DE bag_remove KOLOMMEN
#       - BASE NOTEBOOK UPDATEN & MASTER NOTEBOOK HIERIN AANROEPEN (PAPERMILL)
#       - MACHINE LEARNING WEER LATEN WERKEN
#       - DOCUMENTEREN NOTEBOOKS
#       - DOCUMENTEREN README
#       - PARAMETER OPTIMALISATIE SEARCH RUNNEN

adresDataset.enrich_with_bag(bagDataset.data)
adresDataset.enrich_with_personen_features(personenDataset.data)
adresDataset.add_hotline_features(hotlineDataset.data)

Saving version 'download_leegstand_woningId_bag' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisnummer_toevo

Now looping over all address ids that have a link with one or more inhabitants...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
Now looping over all rows in the main dataframe in order to add person information...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
...done!
Saving version 'download_leegstand_woningId_bag_personen' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisnummer_toevo

Saving version 'download_leegstand_woningId_bag_personen_hotline' of dataframe 'adres'.


In [20]:
x = list(bag.columns)
print(sorted(x))

['_gebiedsgerichtwerken_id', '_geom', '_grootstedelijkgebied_id', '_huisletter_ligplaats', '_huisletter_verblijfsobject', '_huisnummer_ligplaats', '_huisnummer_toevoeging_standplaats', '_huisnummer_toevoeging_verblijfsobject', '_huisnummer_verblijfsobject', '_openbare_ruimte_naam_nummeraanduiding', '_openbare_ruimte_naam_verblijfsobject', 'aantal_kamers', 'adres_nummer', 'begin_geldigheid', 'bouwlaag_toegang', 'bouwlagen', 'bron_id_ligplaats', 'bron_id_nummeraanduiding', 'bron_id_standplaats', 'bron_id_verblijfsobject', 'buurt_id', 'date_modified_nummeraanduiding', 'document_mutatie', 'document_nummer', 'eigendomsverhouding_id#01', 'eigendomsverhouding_id#02', 'einde_geldigheid', 'financieringswijze_id#110', 'financieringswijze_id#200', 'financieringswijze_id#201', 'financieringswijze_id#220', 'financieringswijze_id#250', 'financieringswijze_id#271', 'financieringswijze_id#274', 'financieringswijze_id#301', 'financieringswijze_id#373', 'financieringswijze_id#374', 'financieringswijze_i

In [ ]:
###########################################################
## Remove implicit label columns and superfluous columns ##
###########################################################

adres_remove = [# Remove because cols do not exists when melding is received
                    'wzs_update_datumtijd',
                    # Remove because cols do not add extra information.
                    'kmrs',
                    'straatcode',
                    'xref',
                    'yref',
                    'postcode',
                    'wzs_buurtcode_os_2015',
                    'wzs_buurtcombinatiecode_os_2015',
                    'wzs_stadsdeelcode_os_2015',
                    'sttnaam',
                    'hvv_dag_tek', # Empty column
                    'max_vestig_dtm', # Empty column
                    'wzs_22gebiedencode_os_2015', # Empty column
                    'wzs_22gebiedennaam_os_2015', # Empty column
                    'sdl_naam',
                    'pvh_cd',
                    'sbv_code',
                    'sbw_code',
                    'wzs_wijze_verrijking_geo',
                    'wzs_22gebiedencode_2015',
                    'brt_naam',
                    'wzs_buurtnaam_os_2015',
                    'wzs_buurtcombinatienaam_os_2015',
                    'wzs_rayonnaam_os_2015',
                    'wzs_rayoncode_os_2015',
                    'wzs_stadsdeelnaam_os_2015',
                    'wzs_alternatieve_buurtennaam_os_2015',
                    'wzs_alternatieve_buurtencode_os_2015',
                    'hsltr',
                    'wzs_geom',
                    'brt_code',
                    'brtcombi_code',
                    'brtcombi_naam',
                    'sdl_code',
                    'wzs_22gebiedennaam_2015',
                    'wzs_id',
                    'a_dam_bag',
                    'landelijk_bag']

bag_remove = ['einde_geldigheid',               # Only 2 entries in column.
              'verhuurbare_eenheden',           # Only ~2k entries in column.
              'geometrie_ligplaats',            # Needs a lot of processing before being useful.
              'bron_id',                        # Only 2 entries in column.
              'locatie_ingang_id',              # Only 2 entries in column.
              'reden_afvoer_id',                # Only a few entries in column.
              '_gebiedsgerichtwerken_id',       # Superfluous (gebied).
              '_grootstedelijkgebied_id',       # Superfluous (grootstedelijkgebied).
              'buurt_id',                       # Superfluous (buurt).
              # ONDERSTAANDE 4 KOLOMMEN KONDEN EERDER NIET WEG IVM MATCH MET ADRES DATAFRAME.
              # DEZE MOETEN NU WEL WEG, DAAROM WORDT NU HIER ALLES WEGGEHAALD.
              '_openbare_ruimte_naam_nummeraanduiding',          # Superfluous (straatnaam).
              '_huisnummer_nummeraanduiding',                    # Superfluous (huisnummer).
              '_huisletter_nummeraanduiding',                    # Superfluous (huisletter).
              '_huisnummer_toevoeging_nummeraanduiding',         # Superfluous (huisnummer toevoeging).
              'vervallen',                      # Superfluous (all values in col are equal).
              'document_mutatie',               # Not available at time of signal.
              'date_modified',                  # Not available at time of signal.
              'document_nummer',                # Not needed? (Swaan?)
              'status_coordinaat_omschrijving', # Not needed? (Swaan?)
              'type_woonobject_code',           # Not needed? (Swaan?)
              'id_ligplaats',                   # Not needed.
              'landelijk_id_ligplaats',         # Not needed.
              'id_standplaats',                 # Not needed.
              'landelijk_id_standplaats',       # Not needed.
              'id_verblijfsobject',             # Not needed.
              'landelijk_id_verblijfsobject',   # Not needed.
              ]

# Remove adres_id, since this is not a feature we want our algorihtm to try and learn from.
adresDataset.data.drop(columns=adres_remove + bag_remove + ['adres_id'], inplace=True)